# 6. Hand shape image classification

## 6.2 initial learning

In [1]:
import os
import sys
import glob
import numpy as np
from skimage import io
from sklearn import datasets

In [2]:
#define constant
IMAGE_SIZE = 40
COLOR_BYTE = 3
CATEGORY_NUM = 6

In [2]:
#function for getting hand image

def load_handimage(path):
    files = glob.glob(os.path.join(path, '*/*.png'))
    
    #allocate for image and label
    images = np.ndarray((len(files), IMAGE_SIZE,IMAGE_SIZE, COLOR_BYTE),
                       dtype = np.uint8)
    labels = np.ndarray(len(files), dtype=np.int)
    
    for idx, file in enumerate(files):
        #load image
        image = io.imread(file)
        images[idx] = image
        
        #get label name (※dir name represents label)
        label = os.path.split(os.path.dirname(file))[-1]
        labels[idx] = label
    
    #follow other datasets' format in scikit-learn
    flat_data = images.reshape((-1, IMAGE_SIZE*IMAGE_SIZE*COLOR_BYTE))
    images = flat_data.view()
    return datasets.base.Bunch(data=flat_data,
                              target=labels.astype(np.int),
                              target_names=np.arange(CATEGORY_NUM),
                              images=images,
                              DESCR=None)


In [4]:
train_path='./data/my_learn8'
test_path='./data/my_test2'

In [5]:
from sklearn import svm, metrics

#you can download data from ohm HP

#load dataset
train = load_handimage(train_path)
test = load_handimage(test_path)

#instantiate model and fit
classifier = svm.LinearSVC()
classifier.fit(train.data, train.target)

#evaluate perfornace with test data
predicted = classifier.predict(test.data)

print('Accuracy:', metrics.accuracy_score(test.target, predicted))

Accuracy: 0.9166666666666666


In [6]:
train_path='./data/my_learn10'
test_path='./data/other_test2'

In [7]:
#load dataset
train = load_handimage(train_path)
test = load_handimage(test_path)

#instantiate model and fit
classifier = svm.LinearSVC()
classifier.fit(train.data, train.target)

#evaluate perfornace with test data
predicted = classifier.predict(test.data)

print('Accuracy:', metrics.accuracy_score(test.target, predicted))

Accuracy: 0.16666666666666666


## 6.4 use more dataset

### 6.4.2 train and evaluate

In [8]:
import glob
import os

data_dir = './data'

path_list = sorted(glob.glob(os.path.join(data_dir, '*')))
path_list = path_list[:-4]

print(path_list)

['./data/m01', './data/m02', './data/m03', './data/m04', './data/m04c', './data/m05', './data/m06', './data/m07', './data/m08', './data/m09', './data/m10', './data/m11', './data/m12', './data/m13', './data/m14', './data/m15', './data/m16']


In [9]:
import numpy as np

n_test = 4 # number of test directory

'''
# option : randomize directory
indice = np.arange(len(path_list))
np.random.seed(0)
np.random.shuffle(indice)

paths_for_test = sorted(np.array(path_list)[indice[:n_test]])
paths_for_train = sorted(np.array(path_list)[indice[n_test:]])
'''

#set directories explicitly to follow textbook
paths_for_test =['./data/m01', './data/m05', './data/m06', './data/m07']
paths_for_train =['./data/m02', './data/m03', './data/m04', './data/m08', './data/m09', './data/m10', 
                './data/m11', './data/m12', './data/m13', './data/m14', './data/m15', './data/m16']

print('test ', paths_for_test)
print('train', paths_for_train)

test  ['./data/m01', './data/m05', './data/m06', './data/m07']
train ['./data/m02', './data/m03', './data/m04', './data/m08', './data/m09', './data/m10', './data/m11', './data/m12', './data/m13', './data/m14', './data/m15', './data/m16']


In [10]:
# prepare train dataset
train_data = []
train_label = []

for path in paths_for_train:
    d = load_handimage(path)
    train_data.append(d.data)
    train_label.append(d.target)

train_data = np.concatenate(train_data)
train_label = np.concatenate(train_label)

In [11]:
#classify by SVM
from sklearn import svm, metrics

#instantiate model
classifier = svm.LinearSVC()

#fit
classifier.fit(train_data, train_label)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

### 6.4.3 discussion

In [15]:
# evaluate performance of model with each test dataset

for path in paths_for_test:
    d = load_handimage(path)
    predicted = classifier.predict(d.data)
    
    print('### {0} ###'.format(path))
    print('Accuracy:', metrics.accuracy_score(d.target, predicted))
    print('Classification_Report:\n{0}\n'.format(metrics.classification_report(d.target, predicted)))


### ./data/m01 ###
Accuracy: 0.7283333333333334
Classification_Report:
             precision    recall  f1-score   support

          0       0.76      0.97      0.85       100
          1       0.61      0.45      0.52       100
          2       0.59      0.47      0.52       100
          3       0.67      0.62      0.65       100
          4       0.72      0.86      0.78       100
          5       0.94      1.00      0.97       100

avg / total       0.71      0.73      0.71       600


### ./data/m05 ###
Accuracy: 0.635
Classification_Report:
             precision    recall  f1-score   support

          0       0.61      1.00      0.76       100
          1       0.55      0.42      0.48       100
          2       0.63      0.31      0.42       100
          3       0.59      0.85      0.70       100
          4       0.62      0.72      0.66       100
          5       1.00      0.51      0.68       100

avg / total       0.67      0.64      0.61       600


### ./data/m06 

## 6.5 data cleansing

### 6.5.1 dive into data

##  <font color="Red">original part : follow the discussion on textbook with working code</font>

 ### check : the reason why data/m01 got better performance than others.

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt

num_images = 50

#uncomment below to show test dataset 
'''
for path in paths_for_test:
    d = load_handimage(path)
    images_2 = d.images[d.target==2]
    images_2 = images_2.reshape(-1 ,IMAGE_SIZE, IMAGE_SIZE, COLOR_BYTE)
    
    print('====== {0} ======'.format(path))
    
    fig = plt.figure(figsize=(40,40),dpi=100)
    
    for i, image in enumerate(images_2[:num_images]):
    
        plt.subplot(num_images/5,5,i+1)
        plt.imshow(image)
        
        ax = plt.gca()
        ax.tick_params(labelbottom=False, bottom=False)
        ax.tick_params(labelleft=False, left=False)
        
    plt.tight_layout()
    plt.show()
''' 

"\nfor path in paths_for_test:\n    d = load_handimage(path)\n    images_2 = d.images[d.target==2]\n    images_2 = images_2.reshape(-1 ,IMAGE_SIZE, IMAGE_SIZE, COLOR_BYTE)\n    \n    print('====== {0} ======'.format(path))\n    \n    fig = plt.figure(figsize=(40,40),dpi=100)\n    \n    for i, image in enumerate(images_2[:num_images]):\n    \n        plt.subplot(num_images/5,5,i+1)\n        plt.imshow(image)\n        \n        ax = plt.gca()\n        ax.tick_params(labelbottom=False, bottom=False)\n        ax.tick_params(labelleft=False, left=False)\n        \n    plt.tight_layout()\n    plt.show()\n"

### result : images in data/m01 tend to be more regulated than others.

### 16-fold cross validation

In [17]:
from sklearn import svm, metrics
from sklearn.model_selection import cross_val_score

# prepare train dataset
train_data = []
train_label = []

#set list of path explicitly
path_list =['./data/m01', './data/m02', './data/m03', './data/m04', './data/m05', './data/m06', './data/m07',
            './data/m08', './data/m09', './data/m10', './data/m11', './data/m12', './data/m13', './data/m14',
            './data/m15', './data/m16']
groups = []

for i, path in enumerate(path_list):
    d = load_handimage(path)
    train_data.append(d.data)
    train_label.append(d.target) 
    groups.append([i+1 for _ in range(len(d.data))])
    
train_data = np.concatenate(train_data)
train_label = np.concatenate(train_label)
groups = np.concatenate(groups)

#instantiate model
classifier = svm.LinearSVC()
scores = cross_val_score(classifier, train_data, train_label, groups=groups, cv=16)
   
print('16 fold cross validation score ')
for path, score in zip(path_list, scores):
    print('{0} : {1}'.format(path, score))


16 fold cross validation score 
./data/m01 : 0.6933333333333334
./data/m02 : 0.7066666666666667
./data/m03 : 0.7216666666666667
./data/m04 : 0.355
./data/m05 : 0.6583333333333333
./data/m06 : 0.6933333333333334
./data/m07 : 0.6183333333333333
./data/m08 : 0.6916666666666667
./data/m09 : 0.5116666666666667
./data/m10 : 0.495
./data/m11 : 0.42833333333333334
./data/m12 : 0.625
./data/m13 : 0.6966666666666667
./data/m14 : 0.6566666666666666
./data/m15 : 0.595
./data/m16 : 0.5833333333333334


### test data ./data/m04 got significantly low accuracy.

### check : confusion matrix when using ./data/m04 as test data

In [20]:
from sklearn import svm, metrics

#prepare train dataset
train_data = []
train_label = []

#set list of path explicitly
path_for_train =['./data/m01', './data/m02', './data/m03', 
                 './data/m05', './data/m06', './data/m07', './data/m08',
                 './data/m09', './data/m10', './data/m11', './data/m12',
                 './data/m13', './data/m14','./data/m15', './data/m16']

for path in path_for_train:
    d = load_handimage(path)
    train_data.append(d.data)
    train_label.append(d.target) 
    
train_data = np.concatenate(train_data)
train_label = np.concatenate(train_label)

#prepare test dataset
path_for_test = './data/m04'    

d = load_handimage(path_for_test)
test_data = d.data
test_label = d.target 

#instantiate model
classifier = svm.LinearSVC()

#train
classifier.fit(train_data, train_label)

#predict
predicted = classifier.predict(test_data)

print('Confusion Matrix\n', metrics.confusion_matrix(test_label, predicted))

Confusion Matrix
 [[73 26  0  0  1  0]
 [62 36  1  0  1  0]
 [30 27  9 21 13  0]
 [22 20 12 29 17  0]
 [43 18  7  0 30  2]
 [31  4 10 22  7 26]]


### label '2' got row performance(3 also, not written in textbook)

### 6.5.2 clearning data

In [12]:
#./data/m04 -> ./data/m04c remove noisy data

#set directories explicitly to follow textbook
paths_for_test =['./data/m01', './data/m05', './data/m06', './data/m07']
paths_for_train =['./data/m02', './data/m03', './data/m04c', './data/m08', './data/m09', './data/m10', 
                './data/m11', './data/m12', './data/m13', './data/m14', './data/m15', './data/m16']

# prepare train dataset
train_data = []
train_label = []

for path in paths_for_train:
    d = load_handimage(path)
    train_data.append(d.data)
    train_label.append(d.target)

train_data = np.concatenate(train_data)
train_label = np.concatenate(train_label)

In [13]:
#classify by SVM
from sklearn import svm, metrics

#instantiate model
classifier = svm.LinearSVC()

#fit
classifier.fit(train_data, train_label)


# evaluate performance of model with each test dataset

for path in paths_for_test:
    d = load_handimage(path)
    predicted = classifier.predict(d.data)
    
    print('### {0} ###'.format(path))
    print('Accuracy:', metrics.accuracy_score(d.target, predicted))
    print('Classification_Report:\n{0}\n'.format(metrics.classification_report(d.target, predicted)))

### ./data/m01 ###
Accuracy: 0.7466666666666667
Classification_Report:
             precision    recall  f1-score   support

          0       0.77      0.98      0.86       100
          1       0.67      0.47      0.55       100
          2       0.62      0.58      0.60       100
          3       0.71      0.63      0.67       100
          4       0.73      0.82      0.77       100
          5       0.94      1.00      0.97       100

avg / total       0.74      0.75      0.74       600


### ./data/m05 ###
Accuracy: 0.68
Classification_Report:
             precision    recall  f1-score   support

          0       0.63      1.00      0.77       100
          1       0.68      0.50      0.58       100
          2       0.59      0.70      0.64       100
          3       0.64      0.66      0.65       100
          4       0.73      0.66      0.69       100
          5       1.00      0.56      0.72       100

avg / total       0.71      0.68      0.68       600


### ./data/m06 #

### overall performance got better, but class '2' in ./data/m06 got quite low performance

## 6.6 introduce HOG feature

In [1]:
import os
import sys
import glob
import numpy as np
from skimage import io
from sklearn import datasets
from skimage.feature import hog

In [8]:
#function for getting hand image
#image converted to hog feature 

'''
description:

IMAGE_HEIGHT(H) = 40
IMAGE_WIDTH(W) = 40
CELL_SIZE(C) = 5
BLOCK_SIZE(B) = 5
ORIENTATION(O) = 9

HOG feature is calculated by each block.In each image, the number of block is 
(H/C - B + 1)*(W/C - B + 1) = 16

Each block has 25 cells, and each cell contains HOG feature with 9 orientation.
(0~180 deg, divided by 20)

So, the entire dimension of HOG feature is 
16 * 25 * 9 = 3600
'''

CATEGORY_NUM = 6

def load_handimage(path):
    files = sorted(glob.glob(os.path.join(path, '*/*.png')))
    
    #allocate for image and label
    hogs = np.ndarray((len(files), 3600), dtype = np.float)
    labels = np.ndarray(len(files), dtype=np.int)
    
    for idx, file in enumerate(files):
        #load image
        image = io.imread(file)
        #calculate hog feature
        h = hog(image, orientations=9, pixels_per_cell=(5,5), cells_per_block=(5,5))
        hogs[idx] = h
        
        #get label name (※dir name represents label)
        label = os.path.split(os.path.dirname(file))[-1]
        labels[idx] = label
    
    return datasets.base.Bunch(data=hogs,
                              target=labels.astype(np.int),
                              target_names=np.arange(CATEGORY_NUM),
                              DESCR=None)

In [9]:
#set directories explicitly to follow textbook
paths_for_test =['./data/m01', './data/m05', './data/m06', './data/m07']
paths_for_train =['./data/m02', './data/m03', './data/m04', './data/m08', './data/m09', './data/m10', 
                './data/m11', './data/m12', './data/m13', './data/m14', './data/m15', './data/m16']

# prepare train dataset
train_data = []
train_label = []

for path in paths_for_train:
    d = load_handimage(path)
    train_data.append(d.data)
    train_label.append(d.target)
    
train_data = np.concatenate(train_data)
train_label = np.concatenate(train_label)

/Users/Shota_O/.pyenv/versions/anaconda3-5.0.1/envs/python_learning/lib/python3.7/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


In [10]:
#classify by SVM
from sklearn import svm, metrics

#instantiate model
classifier = svm.LinearSVC()

#fit
classifier.fit(train_data, train_label)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [11]:
# evaluate performance of model with each test dataset

for path in paths_for_test:
    d = load_handimage(path)
    predicted = classifier.predict(d.data)
    
    print('### {0} ###'.format(path))
    print('Accuracy:', metrics.accuracy_score(d.target, predicted))
    print('Classification_Report:\n{0}\n'.format(metrics.classification_report(d.target, predicted)))

/Users/Shota_O/.pyenv/versions/anaconda3-5.0.1/envs/python_learning/lib/python3.7/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


### ./data/m01 ###
Accuracy: 0.8216666666666667
Classification_Report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       100
          1       1.00      0.99      0.99       100
          2       0.75      1.00      0.86       100
          3       0.36      0.08      0.13       100
          4       0.59      0.86      0.70       100
          5       1.00      1.00      1.00       100

avg / total       0.78      0.82      0.78       600


### ./data/m05 ###
Accuracy: 0.865
Classification_Report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       100
          1       0.97      1.00      0.99       100
          2       0.61      0.82      0.70       100
          3       0.69      0.40      0.51       100
          4       0.92      0.97      0.95       100
          5       1.00      1.00      1.00       100

avg / total       0.87      0.86      0.86       600


### ./data/m06 